1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수도 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 문서 임베딩 후 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

spltr=RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤 때 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200, # 문서를 자를 때 어느정도 겹치게 할 지 (앞뒤 문맥을 같이 전달해주기 위함)
)

loader=Docx2txtLoader('./tax.docx')
doc_list=loader.load_and_split(text_splitter=spltr)

In [5]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # 임베딩 과정에도 환경변수 필요하므로 불러와줘야함

embedding=OpenAIEmbeddings(model='text-embedding-3-large')

In [6]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name='tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database=PineconeVectorStore.from_documents(doc_list,embedding,index_name=index_name)

In [7]:
query='연봉 5000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs=database.similarity_search(query, k=3)

In [8]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model='gpt-4o')

In [9]:
from langchain import hub

prompt=hub.pull("rlm/rag-prompt")

In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = database.as_retriever(search_kwargs={"k": 3})

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
ai_msg=rag_chain.invoke(query)
ai_msg

'죄송하지만, 현재 제공된 문맥 정보에는 연봉 5000만원인 직장인의 소득세를 계산하기 위한 구체적인 세율이나 공제 정보가 포함되어 있지 않습니다. 한국 소득세는 기본적으로 누진세율 구조를 따르며, 공제 항목에 따라 최종 세액이 달라질 수 있습니다. 정확한 세액을 산정하려면 최신 소득세법을 참조하시거나 국세청 웹사이트를 방문해보시기 바랍니다.'